In [1]:
import pandas as pd
import numpy as np
import json
from datetime import date
import urllib


#Вытащить данные из json по герою
# Kills 	Deaths 	Assists 	KDA 	Avg. KAL 	GPM 	XPM 	Last Hits 	Denies 	LVL 	HD 	TD 	HH 	GS
def get_data_for_Heroe(i):    
    data = [i.get('hero'), i.get('kills'), i.get('deaths'), i.get('assists'), i.get('kda'), i.get('avgKal'), i.get('gpm'), i.get('xpm'),
            i.get('lastHits'), i.get('denies'), i.get('level'), i.get('heroDamage'), i.get('towerDamage'),
            i.get('heroHealing'), i.get('goldSpent'),]
    return data

In [2]:
# # загрузить json с диска (НА РАБОТЫ)
# import codecs
# json_file = codecs.open( "basick.json")  
# dat = json.load(json_file)
# # создать словарь с данными по каждому герою из json 'dat'
# basick_perfomances={}
# for i in dat.get('data'):
#     data = get_data_for_Heroe(i)
#     basick_perfomances.update({i.get('hero'): data})

In [63]:
# загрузить главный ДФ
main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)
#main = main[:3]

In [64]:
print (main.columns)
main[:3]

Index(['match_id', 'start_time', 'radiant_team_id', 'radiant_score',
       'dire_team_id', 'dire_score', 'radiant_win', 'radiant_H1', 'radiant_H2',
       'radiant_H3', 'radiant_H4', 'radiant_H5', 'dire_H1', 'dire_H2',
       'dire_H3', 'dire_H4', 'dire_H5', 'radiant_P1', 'radiant_P2',
       'radiant_P3', 'radiant_P4', 'radiant_P5', 'dire_P1', 'dire_P2',
       'dire_P3', 'dire_P4', 'dire_P5', 'league_name'],
      dtype='object')


,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
0,2897064589,1483677730,1883502.0,23,350190.0,12,True,86.0,8.0,76.0,...,132851371.0,106573901.0,92423451.0,106809101.0,89871557.0,181716137.0,112377459.0,121052479.0,93616251.0,ESL One Genting 2017
1,2897074199,1483678249,3214108.0,8,1375614.0,37,False,21.0,55.0,71.0,...,86726887.0,87196890.0,40547474.0,47434686.0,139876032.0,117281554.0,82327674.0,149486894.0,87012746.0,ESL One Genting 2017
2,2897155244,1483682366,2581813.0,16,2512249.0,26,False,79.0,63.0,28.0,...,180012313.0,116782914.0,152545459.0,102099826.0,87382579.0,103735745.0,38628747.0,86700461.0,86725175.0,ESL One Genting 2017


In [40]:
columns = ['hero', 'kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
           'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']
# создать файлы с KDA героев по каждой дате (с 2017 года используется только патч 7,00+)
year = 2016
for month in range (12, 13):
    for day in range (11, 32):
         # создать правильную ссылку для поиска по всем героям 
        url_heroes =('http://www.datdota.com/api/heroes/performances?patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&'+ 
               "patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&".format(
               day, month, year) + 
               "duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1") 
        # выгрзить json с предыдущей ссылки
        sauce = urllib.request.Request(url_heroes, headers={'User-agent': 'Mozilla/5.0'} )
        dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
        df = pd.DataFrame(dat, columns=columns)
        
        # запись фала на диск
        df.to_csv('../tabel with heroes DATDOTA (kda)/{}-{}-{}.csv'.format(year, month, day))

# Создать основной ДФ для прогнозов по пикам героев в матче и их kda

In [65]:
columns = ['kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
           'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']
# создать основной массив, где будут записаны все матчи (id матча и rda героя)
df_basick_peromances_heroes = pd.DataFrame()#main.loc[0, 'match_id'

# создать список названий колонок всех героев
all_her = main.loc[:,'radiant_H1':'dire_H5'].columns
for index in main.index:
    # ДФ для соединения всех герове матча в один ДФ 
    df_heroe_match = pd.DataFrame()
    # дата матча
    date_match = date.fromtimestamp(main['start_time'][index])
    # загрузить файл с kda рейтингом героев по дате матча
    df_data_tabel_for_heroes = pd.read_csv('../tabel with heroes DATDOTA (kda)/{}-{}-{}.csv'.
                                               format(date_match.year, date_match.month, date_match.day), index_col=0) 
    for her in all_her:
        # вытащить id героя
        id_heroe = main[her][index]

        # создать массив с данными 
        array = df_data_tabel_for_heroes[df_data_tabel_for_heroes['hero'] == id_heroe].drop('hero', axis=1).values
        # создать название колонок для определнного героя
        col = [her + '_' + c for c  in columns]
        # ДФ для героя по матчу
        df_heroe = pd.DataFrame(array, columns=col)
        df_heroe_match = pd.merge(df_heroe_match, df_heroe, 
                                               left_index=True, right_index=True, how='outer')
    df_basick_peromances_heroes = pd.concat([df_basick_peromances_heroes, df_heroe_match])
df_basick_peromances_heroes = df_basick_peromances_heroes.reset_index().drop('index', axis=1)
df_basick_peromances_heroes['match_id'] = main['match_id']

In [68]:
df_basick_peromances_heroes.to_csv('../tabel/table from Datdota/tabel with KDA heroes for match.csv')

# Сбор данных и создание таблиц по героям, как ими отыгрывает команда

In [75]:
# два ДФ для героев по патчам  в целом и для героев по каждой команде и создать в низ столбец с id матча
df_basick_peromances_heroes = pd.DataFrame()
df_basick_peromances_heroes_teams = pd.DataFrame()
df_basick_peromances_heroes['match_id'] = main['match_id']
df_basick_peromances_heroes_teams['match_id'] = main['match_id']

# создать ДФ с id матчя, пиками героев и фичами по каждому героя для команды
main
for index in main.index:
    # взять одну строчку
    one_match = main.loc[[index]]
    # достать дату матча
    date_match = date.fromtimestamp(one_match['start_time'][index])
    
    # id команды из данных по матчу
    id_team_radiant = int(one_match['radiant_team_id'][index])
    id_team_dire = int(one_match['dire_team_id'][index])
    
   # создать правильную ссылку для посика героев по команде radiant
    url_heroes_team_radiant = ('http://www.datdota.com/api/heroes/performances?teams=1883502&patch=7.09&patch=7.08&patch=7.07&'.format(
                        id_team_radiant) + 'patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&' + 
                       'after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&duration-value-from=0&'.format(
                       date_match.day, date_match.month, date_match.year) + 
                       'duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1')
    # создать правильную ссылку для посика героев по команде dire
    url_heroes_team_dire = ('http://www.datdota.com/api/heroes/performances?teams=1883502&patch=7.09&patch=7.08&patch=7.07&'.format(
                        id_team_dire) + 'patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&' + 
                       'after=01%2F01%2F2011&before={}%2F{}%2F{}&duration=0%3B200&duration-value-from=0&'.format(
                       date_match.day, date_match.month, date_match.year) + 
                       'duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1')
    
    # выгрзить json с предыдущей ссылки
    sauce = urllib.request.Request(url_heroes_team_radiant, headers={'User-agent': 'Mozilla/5.0'})
    dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
    df_team_hero_kda = pd.DataFrame(dat, columns=columns)
    
    # пройтись по всем героям в матче
    for hero in one_match.loc[index, 'radiant_H1':'dire_H5'].columns:
        # взять конкретного героя из ДФ с героями по команде (созданного ранее)
        dat_hero = df_team_hero_kda[df_team_hero_kda['hero'] == one_match[hero]]
        dat_hero
    
    
#     basick_perfomances = {}
#     for i in dat:
#         data = get_data_for_Heroe(i)
#         basick_perfomances.update({i.get('hero'): data})


In [70]:
url_heroes_team_radiant

'http://www.datdota.com/api/heroes/performances?teams=1883502&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011&before=6%2F1%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1'

{'ftp': 'ftp://172.16.0.10:3128',
 'http': 'http://172.16.0.10:3128',
 'https': 'https://172.16.0.10:3128'}

'http://www.datdota.com/api/heroes/performances?teams=1883502&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011&before=6%2F1%2F2017&duration=0%3B200&duration-value-from=0&duration-value-to=200&tier=1&valve-event=does-not-matter&threshold=1'